In [ ]:
import os.path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [ ]:
print(os.path.exists("/content/drive/MyDrive/SS/laptop_price_modf.csv"))

True


In [ ]:
ds = pd.read_csv("/content/drive/MyDrive/SS/laptop_price_modf.csv")

In [ ]:
Y_column = ['Price_euros'] # target variable
X_columns = ds.drop(['Price_euros'], axis=1).columns
X = ds[X_columns]
Y = ds[Y_column]

In [ ]:
# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
# In the first step we will split the data in training and remaining dataset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


(1028, 337)
(1028, 1)
(257, 337)
(257, 1)


In [ ]:
from sklearn import metrics
import pandas as pd

# Створіть функцію для навчання, передбачення та оцінки моделей
def fit_predict_score(model, X_train, Y_train, X_valid, Y_valid):
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_valid)
    return metrics.mean_squared_error(Y_valid, Y_pred), metrics.r2_score(Y_valid, Y_pred)

In [ ]:
ridge_model = Ridge()
mse, r2_score = fit_predict_score(ridge_model, X_train, Y_train, X_test, Y_test)
print("MSE: ", mse)
print("R^2: ", r2_score)
#param_grid = {'alpha': [0.1, 1, 10, 100]}

MSE:  99810.19897733325
R^2:  0.753265582295165


In [ ]:
ridge_model.get_params()

{'alpha': 1.0,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': None,
 'positive': False,
 'random_state': None,
 'solver': 'auto',
 'tol': 0.0001}

In [ ]:
%%time
# Create the random grid
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100],  # Пошук оптимального значення alpha
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],  # Метод розв'язку
}

# print(random_grid)

ridge_model = Ridge()

grid = GridSearchCV(ridge_model, param_grid, cv=5, scoring='r2')

grid.fit(X_train, Y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

The best parameters are {'alpha': 1, 'solver': 'lsqr'} with a score of 0.80
CPU times: user 1min 9s, sys: 14.8 s, total: 1min 24s
Wall time: 1min 16s


In [ ]:
# Найкращі значення гіперпараметрів
best_alpha = grid.best_params_['alpha']
best_solver = grid.best_params_['solver']

# Використання найкращого значення alpha для створення моделі
best_ridge_model = Ridge(alpha=best_alpha, solver=best_solver)

# Пошук значень MSE та R^2 Score для найкращої моделі на тестовій вибірці
mse, r2_score = fit_predict_score(best_ridge_model, X_train, Y_train, X_test, Y_test)

In [ ]:
print("MSE: ", mse)
print("R^2: ", r2_score)

MSE:  99783.79564705757
R^2:  0.753330852281477
